In [1]:
import pandas as pd
import numpy as np

In [17]:
from sklearn.datasets import load_iris

X,y=load_iris(return_X_y=True,as_frame=True)

df=pd.concat([X,y],axis=1)
df=df.sample(frac=1,replace=False)

In [23]:
xx=df.iloc[:,:-1]#.values
y=df.iloc[:,-1]#.values
y.shape,xx.shape

In [36]:
# Defining some global scope variables

target_classes=list(sorted(y.unique()))
x_columns=list(xx.columns)



# First we need to formulate a splitting criterion 
In this case I will be using entropy_values

used to determine how much information is saved by performing splitting
LV divergence based understanding of how far is our probability distribution from the true optimal 1 probability score based distribution 


## Formula used is -summation of i over samples probi*log(prob_i)
## Here samples are our classes and prob i is the number of instances in that class

--> probi(log(Dist true)/log(probi))
-----> Optimal value -probi*log(prob(i))


we can even use gini index for splitting 
summation(pi*(1-pi)) #chossing a class *mistake made in choosing that class

1-summmation(pi2)
This is the measure of statistical dispersion amongsts classes
We want this dispersion to be low 



In [63]:
cls_wts=y.map(
    {0:1,1:3,2:1}
)
cls_wts.name='class_wts'
type(cls_wts)

pandas.core.series.Series

In [68]:
def node_entropy_calculator(class_series,sample_wts=None):
  '''
  This array will only have values of the classes that is present in that node
  If sample weights are present then we will have to group by class and find sum of sample_weights and then perform the further computation

  sample_wts is itself a series

  parameter --> Series
  return entropy value

  [0,1,1,2,0]
  pi*logpi for each of three instances

  '''
  global target_classes

  temp_num=class_series.value_counts().reindex(target_classes)#Series

  if isinstance(sample_wts,type(pd.Series([1]))):
    temp_df=pd.concat([class_series,sample_wts],axis=1)
    temp_num=temp_df.groupby(class_series.name)[sample_wts.name].sum()
    temp_num=temp_num.reindex(target_classes)
  
  temp_num=temp_num[temp_num>0]

  if len(temp_num)==0:
    return 0

  prob_class= temp_num/temp_num.sum()
  log_prob_class=np.log(prob_class)

  entropy=-1*((prob_class*log_prob_class).sum())

  return entropy



def splitter(col_series,split_vaue):
  return



  

# node_entropy_calculator(y)#This is the parent node 0 entropy which we intend to reduce
# node_entropy_calculator(y,cls_wts)



0    True
1    True
dtype: bool

In [71]:
def overall_entropy_calculator(col,split_value,target,sample_wts=None,verbose=0):
  '''
  Will compute and return just the entropy for the classes generated using the provided splitter
  Params--> Column (series), splitting_value val ,target (series)
  return--> Entropy score
  '''
  if isinstance(sample_wts,type(pd.Series([1]))):
    df=pd.concat([col,target,sample_wts],axis=1)
    wt_name=sample_wts.name
  else:
    df=pd.concat([col,target],axis=1)
  
  col_name=col.name
  target_name=target.name
  

  if isinstance(split_value,str):
    lt=df[df[col_name]!=split_value]
    rt=df[df[col_name]==split_value]
  else:
    lt=df[df[col_name]<=split_value]
    rt=df[df[col_name]>split_value]

  if verbose:
    display(rt)
    display(lt)

  if isinstance(sample_wts,type(pd.Series([1]))):
    lt_entropy=node_entropy_calculator(lt[target_name],lt[wt_name])
    rt_entropy=node_entropy_calculator(rt[target_name],rt[wt_name])
  else:
    lt_entropy=node_entropy_calculator(lt[target_name])
    rt_entropy=node_entropy_calculator(rt[target_name])

  n_left=lt.shape[0]
  n_right=rt.shape[0]
  n_total=df.shape[0]

  #Weighted entropy score needs to be computed
  wt_entropy=(n_left/n_total)*(lt_entropy) + (n_right/n_total)*(rt_entropy)
  return wt_entropy




overall_entropy_calculator(xx.iloc[:,0],xx.iloc[:,0].unique()[0],y,
                           sample_wts=cls_wts)


overall_entropy_calculator(xx.iloc[:,0],xx.iloc[:,0].unique()[0],y)



0.9432529919645607

# We will create splits for all possible combinations of column and unique column values


Fn 1-->Check for best split value withing each column
and finally output the placeholder dictionary with 

Fn 2-->Compare entropy values of each column #Comparer that takes dictionary and sorting argument and returns the key of minimum splitter




In [85]:
# entropy_placeholder={}

def detremine_splitter(xx,y):

  for col in xx.columns:
    entropy=np.inf#need to reduce
    for splitter in xx[col].unique():
      temp_entropy=overall_entropy_calculator(
          col=xx[col],split_value=splitter,target=y
      )
      if temp_entropy<entropy:
        entropy=temp_entropy
        col_splitter=(col,splitter,entropy)
        # print(f'splitter updated to {nm}')

  return col_splitter
# detremine_splitter(xx,y)

In [94]:
y[:50].value_counts()#.argmax()

0    19
2    17
1    14
Name: target, dtype: int64

In [137]:
def calculate_leaf_node(y):
  return y.value_counts().argmin()

We need to find the splitter and then split the dataset into right and left datasets and perform the splitting on the created datasets 


We can add if and procedural loop as the recuerssion is at the end 
And we have tail recurssion

In [138]:
class node:
  def __init__(self,val):
    self.val=val
    self.left=None
    self.right=None

dt=node(-1)



# idx

In [140]:
def d_tree(xx,y,max_depth,dt,direction=''):
  if xx.shape[0]==0:
    return
  if max_depth==0:
    print(calculate_leaf_node(y))
    dt.val=('leaf',calculate_leaf_node(y))
    return 
  
  col,split,_=detremine_splitter(xx,y)
  dt.val=(col,split,direction,_)
  print(dt.val)
  dt.left=node(-1)
  dt.right=node(-1)


  idx_rt=xx[xx[col]>split].index
  idx_lt=xx[xx[col]<=split].index

  xx_rt=xx.loc[idx_rt]
  xx_lt=xx.loc[idx_lt]

  y_rt=y.loc[idx_rt]
  y_lt=y.loc[idx_lt]

  d_tree(xx_lt,y_lt,max_depth-1,dt.left,direction='<=')
  d_tree(xx_rt,y_rt,max_depth-1,dt.right,direction='>')



d_tree(xx,y,2,dt=dt)



('petal width (cm)', 0.6, '', 0.46209812037329684)
('petal width (cm)', 0.2, '<=', -0.0)
0
0
('petal width (cm)', 1.7, '>', 0.2147644654371359)
1
1


In [135]:
dt.left.val,dt.val

(('petal width (cm)', 0.2, '<=', -0.0),
 ('petal width (cm)', 0.6, '', 0.46209812037329684))

In [148]:
def BFS(root):
  placeholder=[]
  placeholder.append(root)

  while len(placeholder)>0:
    temp_root=placeholder.pop(0)
    print(temp_root.val)

    if temp_root.left:
      placeholder.append(temp_root.left)
    if temp_root.right:
      placeholder.append(temp_root.right)
    
BFS(dt)

('petal width (cm)', 0.6, '', 0.46209812037329684)
('petal width (cm)', 0.2, '<=', -0.0)
('petal width (cm)', 1.7, '>', 0.2147644654371359)
('leaf', 0)
('leaf', 0)
('leaf', 1)
('leaf', 1)


2

[1, 3]